# mAP

Compute mAP and all related values to it based upon [this](https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173) blog. Related values are:

- precision
- recall
- average precision

In [ ]:
import numpy as np

In [86]:
X = np.array([
    [1, True],
    [2, True],
    [3, False],
    [4, False],
    [5, False],
    [6, True],
    [7, True],
    [8, False],
    [9, False],
    [10, True]
])
X

array([[ 1,  1],
       [ 2,  1],
       [ 3,  0],
       [ 4,  0],
       [ 5,  0],
       [ 6,  1],
       [ 7,  1],
       [ 8,  0],
       [ 9,  0],
       [10,  1]])

In [87]:
# compute precision array
arr = []
count = 0
for idx, x in enumerate(X):
    if x[1] == 1:
        count += 1.
    arr.append(count / (idx+1))
precision_arr = np.array(arr)
precision_arr

array([1.        , 1.        , 0.66666667, 0.5       , 0.4       ,
       0.5       , 0.57142857, 0.5       , 0.44444444, 0.5       ])

In [88]:
# compute recall array
pos_labels = 5
arr = []
count = 0
for x in X:
    if x[1] == 1:
        count += 1.
    arr.append(count / pos_labels)
recall_arr = np.array(arr)
recall_arr

array([0.2, 0.4, 0.4, 0.4, 0.4, 0.6, 0.8, 0.8, 0.8, 1. ])

In [89]:
# combine all data
x2 = np.array([X[:,0], X[:,1], precision_arr, recall_arr])
x2

array([[ 1.        ,  2.        ,  3.        ,  4.        ,  5.        ,
         6.        ,  7.        ,  8.        ,  9.        , 10.        ],
       [ 1.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  1.        ,  0.        ,  0.        ,  1.        ],
       [ 1.        ,  1.        ,  0.66666667,  0.5       ,  0.4       ,
         0.5       ,  0.57142857,  0.5       ,  0.44444444,  0.5       ],
       [ 0.2       ,  0.4       ,  0.4       ,  0.4       ,  0.4       ,
         0.6       ,  0.8       ,  0.8       ,  0.8       ,  1.        ]])

In [90]:
# average precision
prev_max = None
arr = []
bins = np.arange(0.0, 1.1, .1)
for idx, i in enumerate(reversed(bins)):
    try:
        prev_max = np.where(x2[3, :]==i)[0][0]
    except IndexError:
        pass
    arr.append(precision_arr[prev_max])
avg_prec_arr = np.array([x for x in reversed(arr)])
avg_prec_arr

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       0.57142857, 0.57142857, 0.57142857, 0.57142857, 0.5       ,
       0.5       ])

In [91]:
# mAP
np.sum(avg_prec_arr) / len(avg_prec_arr)

0.7532467532467532